In [1]:
#| default_exp core

# core

> Fill in a module description here

In [2]:
#| export
from fastcore.script import *
from pathlib import Path
import polars as pl
import gensim
from gensim.models import Word2Vec
import pandas as pd

In [3]:
#| export
def get_docs(
    dir_comments: str, # directory containing parquet dataframes
    subreddit: str # subreddit to be processed
    ) -> list:
    dir_comments = Path(dir_comments)
    df = pl.read_parquet(dir_comments / subreddit / '*.parquet')
    print('finished reading parquet files')
    df = df.sample(10_000)
    docs_srs = df['body'].str.split(' ')
    print('finished tokenizing')
    docs_list = docs_srs.to_list()
    print('finished converting to list')
    # return docs_list
    return docs_srs
    

In [20]:
#| notest
docs = get_docs('/Users/quirin/proj/getreddit/out/', 'Coronavirus')

finished reading parquet files
finished tokenizing
finished converting to list


In [5]:
fpaths = list(Path('/Users/quirin/proj/getreddit/out/Coronavirus').glob('*.parquet'))

In [6]:
df = pd.read_parquet(fpaths[:1])

In [7]:
docs = df['body'].str.split(' ')

In [8]:
docs

0                                                     [Yes]
1         [Just, eat, raw, elk, meat, and, work, out, su...
2         [They, had, the, Big, Pendleton, concert, whic...
3         [Agreed,, and, I, don’t, understand, this, at,...
4         [Some?, \n\n#SOME, vaccinated, Americans?, \n\...
                                ...                        
508550    [They, need, more, testing., Seems, like, them...
508551                                      [Oh, my, gosh!]
508552    [Among, documented, cases., We're, still, mass...
508553                                          [[removed]]
508554                                          [[removed]]
Name: body, Length: 508555, dtype: object

In [14]:
model = Word2Vec(
    docs[:10_000],
    workers=8,
    callbacks=[my_callback],
    min_count=5,
    window=5,
    epochs=5,
    vector_size=300,
    batch_words=10_000
)

In [15]:
# get number of words in model
len(model.wv.key_to_index)

4712

In [16]:
model.wv.most_similar('think')

[('know', 0.9908594489097595),
 ('understand', 0.9546481966972351),
 ('say', 0.9420730471611023),
 ('feel', 0.9382495284080505),
 ('care', 0.9364539980888367),
 ('don’t', 0.9326096177101135),
 ("don't", 0.9324885010719299),
 ('mean', 0.929454505443573),
 ('like,', 0.9275861978530884),
 ('what', 0.9259113073348999)]

# train model

In [47]:
#| export
class Corpus:
    """An iterator that yields sentences (lists of str)."""
    def __init__(self, docs):
        self.docs_clean = docs

    def __iter__(self):
        for doc in self.docs_clean:
            yield doc

In [48]:
corpus = Corpus(docs)

In [10]:
#| export
class MyCallback(gensim.models.callbacks.CallbackAny2Vec):
    # Initialize any variables or attributes here
    def __init__(self):
        self.epoch = 0 # Keep track of the current epoch number

    # Do something at the start of each epoch
    def on_epoch_begin(self, model):
        logging.info(f"Epoch {self.epoch} started")

    # Do something at the end of each epoch
    def on_epoch_end(self, model):
        logging.info(f"Epoch {self.epoch} finished")
        self.epoch += 1 # Increment the epoch number

    # Do something at the start of each batch
    def on_batch_begin(self, model):
        pass # You can add your own code here

    # Do something at the end of each batch
    def on_batch_end(self, model, cumulative_stats):
        # Get some statistics from cumulative_stats dictionary
        total_examples = cumulative_stats['total_examples']
        total_words = cumulative_stats['total_words']
        job = cumulative_stats['job']
        raw_words = cumulative_stats['raw_words']
        effective_words = cumulative_stats['effective_words']

        # Calculate and print some percentages using these statistics
        percentage_sentences = (job[1] - job[0]) / total_examples * 100
        percentage_words = raw_words / total_words * 100
        percentage_effective_words = effective_words / raw_words * 100

        logging.info(f"Batch processed {percentage_sentences:.2f}% sentences and {percentage_words:.2f}% words")
        logging.info(f"Batch used {percentage_effective_words:.2f}% words effectively for training")

In [13]:
#| export
import logging

In [11]:
my_callback = MyCallback()

In [52]:
#| export
@call_parse
def train_model(
    subreddit: str, # Subreddit to be processed
    dir_comments: str, # Directory containing parquet dataframes
    dir_models: str # Directory to save model
    ) -> Word2Vec:
    """
    Trains a word2vec model on the comments of a subreddit.
    """
    docs = get_docs(dir_comments, subreddit)
    corpus = Corpus(docs)
    my_callback = MyCallback()
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    model = Word2Vec(
        corpus,
        workers=8,
        callbacks=[my_callback],
        min_count=5,
        window=5,
        epochs=5,
        vector_size=300,
        batch_words=10_000
    )
    dir_models = Path(dir_models)
    model.save((dir_models / f"{subreddit}.model").as_posix())
    return model

In [ ]:
#| notest
model = train_model('conspiracy', '~/proj/getreddit/out/', '~/proj/redditdumps/out/')

# toy console script

In [ ]:
#| export
@call_parse
def add_one(
    num: int # first number
    ) -> int: # result
    result = num + 1
    print(result)
    return result